# Titanic : Machine Learning from Disaster

### 1. Importing in Dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("train.csv")
data.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
805,806,0,3,"Johansson, Mr. Karl Johan",male,31.0,0,0,347063,7.7750,NaN,S
82,83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q
736,737,0,3,"Ford, Mrs. Edward (Margaret Ann Watson)",female,48.0,1,3,W./C. 6608,34.3750,NaN,S


In [3]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 2. Transforming Features

In [4]:
def simplify_ages(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 100)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenage', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df.Age, bins, labels=group_names)
    df.Age = categories
    return df

In [5]:
def simplify_cabins(df):
    df.Cabin = df.Cabin.fillna('E')
    df.Cabin = df.Cabin.apply(lambda x: x[0])
    return df

In [6]:
def simplify_fares(df):
    df.Fare = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df.Fare, bins, labels=group_names)
    df.Fare = categories
    return df

In [7]:
def simplify_embarked(df):
    df.Embarked = df.Embarked.fillna('S')
    return df

In [8]:
def format_name(df):
    df['Lname'] = df.Name.apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df.Name.apply(lambda x: x.split(' ')[1])
    return df  

In [9]:
def drop_features(df):
    return df.drop(['Ticket', 'Name'], axis=1)

In [10]:
def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = simplify_embarked(df)
    df = format_name(df)
    df = drop_features(df)
    return df

In [11]:
data_train = transform_features(data)
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Lname,NamePrefix
0,1,0,3,male,Student,1,0,1_quartile,E,S,"Braund,",Mr.
1,2,1,1,female,Adult,1,0,4_quartile,C,C,"Cumings,",Mrs.
2,3,1,3,female,Young Adult,0,0,1_quartile,E,S,"Heikkinen,",Miss.
3,4,1,1,female,Young Adult,1,0,4_quartile,C,S,"Futrelle,",Mrs.
4,5,0,3,male,Young Adult,0,0,2_quartile,E,S,"Allen,",Mr.


In [46]:
X = data_train.iloc[:, 2:12].values
y = data_train.iloc[:, 1].values

### 3. Encoding

In [47]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [48]:
column_no = [0,1,2,5,6,7,8,9]
labelencoder_X = LabelEncoder()

for no in column_no:
    X[:,no] = labelencoder_X.fit_transform(X[:,no])

In [15]:
onehotencoder = OneHotEncoder(categorical_features = [0,2,5,6,7,8,9])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]

### 4. Machine Learning

In [49]:
#splitting the dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 5. Feature scaling

In [45]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

/home/agent004/anaconda2/envs/dlAZ/lib/python3.5/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [50]:
X_train.shape

(712, 10)

### 6. Creating ANN

In [51]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [58]:
#Initaializing the ANN
classifier = Sequential()

#Adding the input layer and the first hidden layer with dropout
classifier.add(Dense(activation="relu", kernel_initializer="uniform", input_dim=10, units=6))
#classifier.add(Dropout(rate = 0.1))

#Adding the second hidden layer
classifier.add(Dense(activation="relu", kernel_initializer="uniform", units=6))
#classifier.add(Dropout(rate = 0.1))

#Adding the output layer
classifier.add(Dense(activation="sigmoid", kernel_initializer="uniform", units=1))

#Compiling the ANN
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [59]:
#Fitting the ANN to training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
712/712 [==============================] - 0s - loss: 0.6830 - acc: 0.6081     
Epoch 2/100
712/712 [==============================] - 0s - loss: 0.6666 - acc: 0.6166     
Epoch 3/100
712/712 [==============================] - 0s - loss: 0.6648 - acc: 0.6166     
Epoch 4/100
712/712 [==============================] - 0s - loss: 0.6648 - acc: 0.6166     
Epoch 5/100
712/712 [==============================] - 0s - loss: 0.6637 - acc: 0.6166     
Epoch 6/100
712/712 [==============================] - 0s - loss: 0.6661 - acc: 0.6166     
Epoch 7/100
712/712 [==============================] - 0s - loss: 0.6662 - acc: 0.6166     
Epoch 8/100
712/712 [==============================] - 0s - loss: 0.6608 - acc: 0.6166     
Epoch 9/100
712/712 [==============================] - 0s - loss: 0.6596 - acc: 0.6166     
Epoch 10/100
712/712 [==============================] - 0s - loss: 0.6572 - acc: 0.6166     
Epoch 11/100
712/712 [==============================] - 0s - loss: 0.6539 - acc

712/712 [==============================] - 0s - loss: 0.4899 - acc: 0.7893     
Epoch 89/100
712/712 [==============================] - 0s - loss: 0.4886 - acc: 0.7893     
Epoch 90/100
712/712 [==============================] - 0s - loss: 0.4901 - acc: 0.7809     
Epoch 91/100
712/712 [==============================] - 0s - loss: 0.4831 - acc: 0.7921     
Epoch 92/100
712/712 [==============================] - 0s - loss: 0.4873 - acc: 0.7935     
Epoch 93/100
712/712 [==============================] - 0s - loss: 0.4864 - acc: 0.7963     
Epoch 94/100
712/712 [==============================] - 0s - loss: 0.4793 - acc: 0.7992     
Epoch 95/100
712/712 [==============================] - 0s - loss: 0.4797 - acc: 0.7921     
Epoch 96/100
712/712 [==============================] - 0s - loss: 0.4845 - acc: 0.7992     
Epoch 97/100
712/712 [==============================] - 0s - loss: 0.4891 - acc: 0.7851     
Epoch 98/100
712/712 [==============================] - 0s - loss: 0.4821 - acc: 0.

In [60]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [61]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [62]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(activation="relu", kernel_initializer="uniform", input_dim=10, units=5))
    #classifier.add(Dropout(0.1))
    classifier.add(Dense(activation="relu", kernel_initializer="uniform", units=5))
    #classifier.add(Dropout(0.1))
    classifier.add(Dense(activation="sigmoid", kernel_initializer="uniform", units=1))
    classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])
    return classifier

In [63]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)

In [65]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1, verbose=1)


Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100


KeyboardInterrupt: 